In [2]:
# import libraries
from bs4 import BeautifulSoup
import requests
from urllib.request import (urlopen, urlparse, urlunparse, urlretrieve)
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np 
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
from urllib.request import (Request, urlopen, urlparse, urlunparse, urlretrieve)

print('All libraries loaded ...')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.1.0-py_0 conda-forge
    branca:  0.3.0-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge

branca-0.3.0-p 100% |################################| Time: 0:00:00 325.74 kB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00 377.02 kB/s
altair-2.1.0-p 100% |################################| Time: 0:00:00 420.24 kB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00 620.42 kB/s
All libraries loaded ...


In [4]:
json_request = 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/united_states_rankings_2017_limit0_068d27a033ada75528617769a8a4c4c2.json'
response = urlopen(json_request)

# Convert bytes to string type and string type to dict

string = response.read().decode('utf-8')
json_obj = json.loads(string)


In [5]:
univ_df = json_normalize(json_obj['data'])

In [6]:
pd.set_option('max_colwidth', 200)
pd.set_option('max_columns',100)
univ_df.head()

,aliases,apply_link,location,member_level,name,nid,rank,rank_order,record_type,scores_engagement,scores_engagement_rank,scores_environment,scores_environment_rank,scores_outcomes,scores_outcomes_rank,scores_overall,scores_overall_rank,scores_resources,scores_resources_rank,stats_board,stats_fees_oos,stats_salary,subjects_offered,url
0,Stanford University,NaN,California,11,Stanford University,467,1,1,private,17.4,38,7.9,66,39.5,6,92.1,1,27.3,7,"$13,631","$44,757","$83,400","Business & Management,General Engineering,Education,Chemistry,Law,Accounting & Finance,Biological Sciences,Economics & Econometrics,Civil Engineering,Geology, Environmental, Earth & Marine Science...",/world-university-rankings/stanford-university
1,Harvard University,NaN,Massachusetts,0,Harvard University,466,2,2,private,15.2,647,6.8,182,39.6,3,91.6,2,29.9,1,"$14,669","$43,938","$91,300","Other Health,Archaeology,Computer Science,Art, Performing Arts & Design,Geology, Environmental, Earth & Marine Sciences,Politics & International Studies (incl Development Studies),Psychology,Chemi...",/world-university-rankings/harvard-university
2,Massachusetts Institute of Technology,NaN,Massachusetts,0,Massachusetts Institute of Technology,471,3,3,private,15.7,471,7.1,138,39.3,9,91.4,3,29.3,3,"$13,224","$45,016","$90,400","Chemistry,Archaeology,Other Health,Accounting & Finance,Electrical & Electronic Engineering,Mechanical & Aerospace Engineering,Architecture,Communication & Media Studies,Chemical Engineering,Art, ...",/world-university-rankings/massachusetts-institute-technology
3,University of Pennsylvania,NaN,Pennsylvania,0,University of Pennsylvania,480,4,4,private,17.6,20,6.9,169,39.6,4,91.3,4,27.2,8,"$13,464","$47,668","$78,900","Physics & Astronomy,Politics & International Studies (incl Development Studies),Psychology,Sociology,Geography,Mathematics & Statistics,Chemistry,Veterinary Science,Computer Science,Mechanical & A...",/world-university-rankings/university-pennsylvania
4,Columbia University,NaN,New York,0,Columbia University,476,5,5,private,16.8,145,7.8,69,39.6,1,91.1,5,27.1,10,"$12,432","$51,008","$74,000","Medicine & Dentistry,Other Health,Communication & Media Studies,General Engineering,Sociology,Law,Mathematics & Statistics,Accounting & Finance,Languages, Literature & Linguistics,Physics & Astron...",/world-university-rankings/columbia-university


In [7]:
univ_df.shape

(1061, 24)

In [8]:

university_df = univ_df.drop(['aliases','apply_link','member_level','nid','rank','url','scores_engagement_rank','scores_environment_rank','scores_outcomes_rank','scores_resources_rank', 'scores_overall_rank', 'subjects_offered'], axis=1).copy()
university_df.shape

(1061, 12)

In [9]:
# The code was removed by Watson Studio for sharing.

In [10]:
# Get Lat & Lng for each University

latitudes = []
longitudes = []

for index, univ in university_df.iterrows():
    url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&components=country:US&address={}'.format(API_key,univ['name'])
    response = requests.get(url).json()
    try:
        geographical_data = response['results'][0]['geometry']['location'] # get geographical coordinates
    except:
        print('no response for', index, univ['name'])
    latitudes.append(geographical_data['lat'])
    longitudes.append(geographical_data['lng'])
    print('index:  {},  university: {}, city: {}, lat: {}, Lng: {}'.format(index,univ['name'],univ['location'],geographical_data['lat'],geographical_data['lng']))


index:  0,  university: Stanford University, city: California, lat: 37.4274745, Lng: -122.169719
index:  1,  university: Harvard University, city: Massachusetts, lat: 42.3770029, Lng: -71.11666009999999
index:  2,  university: Massachusetts Institute of Technology, city: Massachusetts, lat: 42.360091, Lng: -71.09416
index:  3,  university: University of Pennsylvania, city: Pennsylvania, lat: 39.9522188, Lng: -75.1932137
index:  4,  university: Columbia University, city: New York, lat: 40.8075355, Lng: -73.9625727
index:  5,  university: Yale University, city: Connecticut, lat: 41.3163244, Lng: -72.92234309999999
index:  6,  university: Duke University, city: North Carolina, lat: 36.0014258, Lng: -78.9382286
index:  7,  university: Cornell University, city: New York, lat: 42.4534492, Lng: -76.4735027
index:  8,  university: Princeton University, city: New Jersey, lat: 40.3439888, Lng: -74.6514481
index:  9,  university: Northwestern University, city: Illinois, lat: 42.0564594, Lng: -87.

In [11]:
# Append lat & lng columns to the dataframe


lat_df = pd.DataFrame(latitudes, columns=['Latitude'])
lng_df = pd.DataFrame(longitudes, columns=['Longitude'])

final_df = pd.concat([university_df,lat_df,lng_df],axis=1)

In [12]:
final_df.head()

,location,name,rank_order,record_type,scores_engagement,scores_environment,scores_outcomes,scores_overall,scores_resources,stats_board,stats_fees_oos,stats_salary,Latitude,Longitude
0,California,Stanford University,1,private,17.4,7.9,39.5,92.1,27.3,"$13,631","$44,757","$83,400",37.427475,-122.169719
1,Massachusetts,Harvard University,2,private,15.2,6.8,39.6,91.6,29.9,"$14,669","$43,938","$91,300",42.377003,-71.116660
2,Massachusetts,Massachusetts Institute of Technology,3,private,15.7,7.1,39.3,91.4,29.3,"$13,224","$45,016","$90,400",42.360091,-71.094160
3,Pennsylvania,University of Pennsylvania,4,private,17.6,6.9,39.6,91.3,27.2,"$13,464","$47,668","$78,900",39.952219,-75.193214
4,New York,Columbia University,5,private,16.8,7.8,39.6,91.1,27.1,"$12,432","$51,008","$74,000",40.807536,-73.962573


In [15]:
# save dataset
final_df.to_csv('final_df.csv', index=False)


In [16]:
# The code was removed by Watson Studio for sharing.

In [17]:
final_df = pd.read_csv(body)
final_df.head()

,location,name,rank_order,record_type,scores_engagement,scores_environment,scores_outcomes,scores_overall,scores_resources,stats_board,stats_fees_oos,stats_salary,Latitude,Longitude
0,California,Stanford University,1,private,17.4,7.9,39.5,92.1,27.3,"$13,631","$44,757","$83,400",37.427475,-122.169719
1,Massachusetts,Harvard University,2,private,15.2,6.8,39.6,91.6,29.9,"$14,669","$43,938","$91,300",42.377003,-71.116660
2,Massachusetts,Massachusetts Institute of Technology,3,private,15.7,7.1,39.3,91.4,29.3,"$13,224","$45,016","$90,400",42.360091,-71.094160
3,Pennsylvania,University of Pennsylvania,4,private,17.6,6.9,39.6,91.3,27.2,"$13,464","$47,668","$78,900",39.952219,-75.193214
4,New York,Columbia University,5,private,16.8,7.8,39.6,91.1,27.1,"$12,432","$51,008","$74,000",40.807536,-73.962573


In [18]:
# Select the top 200 universities for analysis
df = final_df[:200].copy()
print(df.shape)
df.head()

(200, 14)


,location,name,rank_order,record_type,scores_engagement,scores_environment,scores_outcomes,scores_overall,scores_resources,stats_board,stats_fees_oos,stats_salary,Latitude,Longitude
0,California,Stanford University,1,private,17.4,7.9,39.5,92.1,27.3,"$13,631","$44,757","$83,400",37.427475,-122.169719
1,Massachusetts,Harvard University,2,private,15.2,6.8,39.6,91.6,29.9,"$14,669","$43,938","$91,300",42.377003,-71.116660
2,Massachusetts,Massachusetts Institute of Technology,3,private,15.7,7.1,39.3,91.4,29.3,"$13,224","$45,016","$90,400",42.360091,-71.094160
3,Pennsylvania,University of Pennsylvania,4,private,17.6,6.9,39.6,91.3,27.2,"$13,464","$47,668","$78,900",39.952219,-75.193214
4,New York,Columbia University,5,private,16.8,7.8,39.6,91.1,27.1,"$12,432","$51,008","$74,000",40.807536,-73.962573


In [19]:
# We select 11 most important venue categories (purely subjective) on which to assess overall University allure

venue_cat = ['Office',
'Shop & Service',
'Food',
'Arts & Entertainment',
'Outdoors & Recreation',
'Library',
'College & University',
'Nightlife Spot',
'Residence',
'Government Building',
'Travel & Transport']

# Corresponding Category ID from Foursquare Docs

venue_catid = ['4bf58dd8d48988d124941735',
'4d4b7105d754a06378d81259',
'4d4b7105d754a06374d81259',
'4d4b7104d754a06370d81259',
'4d4b7105d754a06377d81259',
'4bf58dd8d48988d12f941735',
'4d4b7105d754a06372d81259',
'4d4b7105d754a06376d81259',
'4e67e38e036454776db1fb3a',
'4bf58dd8d48988d126941735',
'4d4b7105d754a06379d81259']

cat_name_df = pd.DataFrame(venue_cat, columns=['cat_name'])
cat_id_df =  pd.DataFrame(venue_catid, columns=['cat_id'])
cat_df = pd.concat([cat_name_df,cat_id_df], axis=1)

cat_df

,cat_name,cat_id
0,Office,4bf58dd8d48988d124941735
1,Shop & Service,4d4b7105d754a06378d81259
2,Food,4d4b7105d754a06374d81259
3,Arts & Entertainment,4d4b7104d754a06370d81259
4,Outdoors & Recreation,4d4b7105d754a06377d81259
5,Library,4bf58dd8d48988d12f941735
6,College & University,4d4b7105d754a06372d81259
7,Nightlife Spot,4d4b7105d754a06376d81259
8,Residence,4e67e38e036454776db1fb3a
9,Government Building,4bf58dd8d48988d126941735


In [20]:
# Append Venue Category Columns to the Dataframe & initialize values with NaN

df = pd.concat([df,pd.DataFrame(columns=venue_cat)])

df.head()

,Arts & Entertainment,College & University,Food,Government Building,Latitude,Library,Longitude,Nightlife Spot,Office,Outdoors & Recreation,Residence,Shop & Service,Travel & Transport,location,name,rank_order,record_type,scores_engagement,scores_environment,scores_outcomes,scores_overall,scores_resources,stats_board,stats_fees_oos,stats_salary
0,NaN,NaN,NaN,NaN,37.427475,NaN,-122.169719,NaN,NaN,NaN,NaN,NaN,NaN,California,Stanford University,1.0,private,17.4,7.9,39.5,92.1,27.3,"$13,631","$44,757","$83,400"
1,NaN,NaN,NaN,NaN,42.377003,NaN,-71.116660,NaN,NaN,NaN,NaN,NaN,NaN,Massachusetts,Harvard University,2.0,private,15.2,6.8,39.6,91.6,29.9,"$14,669","$43,938","$91,300"
2,NaN,NaN,NaN,NaN,42.360091,NaN,-71.094160,NaN,NaN,NaN,NaN,NaN,NaN,Massachusetts,Massachusetts Institute of Technology,3.0,private,15.7,7.1,39.3,91.4,29.3,"$13,224","$45,016","$90,400"
3,NaN,NaN,NaN,NaN,39.952219,NaN,-75.193214,NaN,NaN,NaN,NaN,NaN,NaN,Pennsylvania,University of Pennsylvania,4.0,private,17.6,6.9,39.6,91.3,27.2,"$13,464","$47,668","$78,900"
4,NaN,NaN,NaN,NaN,40.807536,NaN,-73.962573,NaN,NaN,NaN,NaN,NaN,NaN,New York,Columbia University,5.0,private,16.8,7.8,39.6,91.1,27.1,"$12,432","$51,008","$74,000"


In [21]:
# Plot the Universities on the Map

address = 'US'

geolocator = Nominatim(user_agent="random_string",timeout=5)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of US are {}, {}.'.format(latitude, longitude))


The geograpical coordinates of US are 39.7837304, -100.4458825.


In [22]:
us_map = folium.Map(location=[latitude, longitude], zoom_start=4)
#us_map

In [23]:
# Show all universities on map
# add markers to map

for lat, lng, univ, city in zip(df['Latitude'], df['Longitude'], df['name'], df['location']):
    label = '{}, {}'.format(univ,city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(us_map)  

us_map

In [24]:
# The code was removed by Watson Studio for sharing.

#### Limiting the Foursquare Venue Search - (radius = 2KM, LIMIT = 30/Category) 

In [25]:
# Create function to loop through University locations and fetch count of venues - upto 30 - for each venue category

def venue_category_count(university, city, latitudes, longitudes, radius):
    # for each University ...
    for univ, city, lat, lng in zip(university, city, latitudes, longitudes):
        print("getting venue counts for: {},{}".format(univ,city))            
        # for each Venue Category
        for index, row in cat_df.iterrows():
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}&intent=browse'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT,
                row['cat_id'])
            #Get the results
            #print("category_id: ",row['cat_id'])
            #print("url: ", url)
            try:
                results = requests.get(url).json()['response']['venues']
                venue_count = len(results)
                print("venue count: ",venue_count)
                df.loc[df['name']==univ,row['cat_name']] = venue_count
            except:
                pass
                results = []
                venue_count = len(results)
                print("venue count - zero response: ",venue_count)
                df.loc[df['name']==univ,row['cat_name']] = venue_count
    


In [31]:
# Populate the venue count for each category in the main dataframe

venue_category_count(df['name'], df['location'], df['Latitude'], df['Longitude'], radius=2000)

getting venue counts for: Stanford University,California
venue count:  29
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  6
venue count:  30
venue count:  7
venue count:  27
venue count:  10
venue count:  16
getting venue counts for: Harvard University,Massachusetts
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  19
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  30
getting venue counts for: Massachusetts Institute of Technology,Massachusetts
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  30
getting venue counts for: University of Pennsylvania,Pennsylvania
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count:  24
venue count:  30
venue count:  30
venue count:  30
venue count:  30
venue count

In [33]:
#df.head(10)
df.tail(10)

,Arts & Entertainment,College & University,Food,Government Building,Latitude,Library,Longitude,Nightlife Spot,Office,Outdoors & Recreation,Residence,Shop & Service,Travel & Transport,location,name,rank_order,record_type,scores_engagement,scores_environment,scores_outcomes,scores_overall,scores_resources,stats_board,stats_fees_oos,stats_salary
190,4,30,9,0,35.205180,0,-85.917660,6,4,21,2,10,4,Tennessee,Sewanee-The University of the South,191,private,15.9,2.8,20.9,60.7,21.1,"$10,600","$35,202","$39,400"
191,14,30,12,4,42.716635,0,-73.752336,15,10,30,0,30,12,New York,Siena College,192,private,14.8,2.6,25.5,60.5,17.6,"$13,060","$32,293","$54,600"
192,30,30,23,4,36.151988,1,-95.946000,30,30,30,29,25,30,Oklahoma,University of Tulsa,193,private,16.9,5.4,16.8,60.5,21.4,"$10,476","$34,405","$43,400"
193,30,22,30,11,39.739300,2,-84.179967,12,30,30,30,19,30,Ohio,University of Dayton,194,private,17.1,4.1,24.3,60.4,14.9,"$11,840","$37,230","$51,400"
194,30,30,27,23,46.070580,1,-118.330635,30,30,30,13,30,30,Washington,Whitman College,195,private,16.0,3.2,18.4,60.4,22.8,"$11,228","$44,800","$43,400"
195,25,30,30,13,40.941889,1,-90.371867,30,26,30,11,30,30,Illinois,Knox College,196,private,15.4,5.7,20.8,60.3,18.3,"$8,724","$40,497","$41,600"
196,3,30,30,0,41.921177,0,-71.537877,7,20,30,4,30,6,Rhode Island,Bryant University,197,private,15.8,4.2,25.7,60.2,14.5,"$14,102","$38,574","$63,200"
197,2,0,4,1,41.305824,0,-95.083325,0,1,4,0,8,1,Oregon,Lewis & Clark College,198,private,14.2,5.0,22.1,60.2,18.9,"$10,906","$43,382","$42,200"
198,0,0,5,0,47.751074,0,-120.740139,0,0,9,0,4,2,Pennsylvania,Washington & Jefferson College,199,private,14.9,2.6,23.9,60.2,18.8,"$10,666","$41,282","$48,900"
199,30,30,30,23,38.936881,3,-76.998692,30,30,30,19,30,30,District of Columbia,Catholic University of America,200,private,16.3,4.4,16.5,60.1,22.9,"$14,518","$39,726","$54,200"


In [34]:
# Convert numeric columns (dollars, scores) to Floats

df['stats_board'] = df['stats_board'].replace('[\$,]', '', regex=True).astype(float)
df['stats_fees_oos'] = df['stats_fees_oos'].replace('[\$,]', '', regex=True).astype(float)
df['stats_salary'] = df['stats_salary'].replace('[\$,]', '', regex=True).astype(float)
df[['scores_engagement','scores_environment','scores_outcomes','scores_overall','scores_resources']] = df[['scores_engagement','scores_environment','scores_outcomes','scores_overall','scores_resources']].astype(float)


In [36]:
df.dtypes

Arts & Entertainment       int64
College & University       int64
Food                       int64
Government Building        int64
Latitude                 float64
Library                    int64
Longitude                float64
Nightlife Spot             int64
Office                     int64
Outdoors & Recreation      int64
Residence                  int64
Shop & Service             int64
Travel & Transport         int64
location                  object
name                      object
rank_order                object
record_type               object
scores_engagement        float64
scores_environment       float64
scores_outcomes          float64
scores_overall           float64
scores_resources         float64
stats_board              float64
stats_fees_oos           float64
stats_salary             float64
dtype: object

In [38]:
# save the final dataframe

df.to_csv('df.csv', index=False)

In [26]:
# read back the saved dataset

body = client_4a88d9dc37ed42bf8d5103d6eac71c97.get_object(Bucket='courseracapstone-donotdelete-pr-l8npvzwkvhkymf',Key='df.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,Arts & Entertainment,College & University,Food,Government Building,Latitude,Library,Longitude,Nightlife Spot,Office,Outdoors & Recreation,Residence,Shop & Service,Travel & Transport,location,name,rank_order,record_type,scores_engagement,scores_environment,scores_outcomes,scores_overall,scores_resources,stats_board,stats_fees_oos,stats_salary
0,30,30,30,10,37.427475,6,-122.169719,7,29,30,27,30,16,California,Stanford University,1,private,17.4,7.9,39.5,92.1,27.3,13631.0,44757.0,83400.0
1,30,30,30,30,42.377003,19,-71.116660,30,30,30,30,30,30,Massachusetts,Harvard University,2,private,15.2,6.8,39.6,91.6,29.9,14669.0,43938.0,91300.0
2,30,30,30,30,42.360091,30,-71.094160,30,30,30,30,30,30,Massachusetts,Massachusetts Institute of Technology,3,private,15.7,7.1,39.3,91.4,29.3,13224.0,45016.0,90400.0
3,30,30,30,30,39.952219,24,-75.193214,30,30,30,30,30,30,Pennsylvania,University of Pennsylvania,4,private,17.6,6.9,39.6,91.3,27.2,13464.0,47668.0,78900.0
4,30,30,30,14,40.807536,23,-73.962573,30,24,30,30,30,30,New York,Columbia University,5,private,16.8,7.8,39.6,91.1,27.1,12432.0,51008.0,74000.0


#### Create a new DataFrame with one hot encoded variables for Private, Public Universities

In [27]:

df_onehot = pd.get_dummies(df['record_type'])
df_onehot[venue_cat] = df[venue_cat]
df_onehot[['scores_engagement','scores_environment','scores_outcomes','scores_resources']] = df[['scores_engagement','scores_environment','scores_outcomes','scores_resources']]
df_onehot[['stats_board','stats_fees_oos','stats_salary']] = df[['stats_board','stats_fees_oos','stats_salary']]
df_onehot[['name']] = df[['name']]


In [28]:
df_onehot

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,name
0,1,0,29,30,30,30,30,6,30,7,27,10,16,17.4,7.9,39.5,27.3,13631.0,44757.0,83400.0,Stanford University
1,1,0,30,30,30,30,30,19,30,30,30,30,30,15.2,6.8,39.6,29.9,14669.0,43938.0,91300.0,Harvard University
2,1,0,30,30,30,30,30,30,30,30,30,30,30,15.7,7.1,39.3,29.3,13224.0,45016.0,90400.0,Massachusetts Institute of Technology
3,1,0,30,30,30,30,30,24,30,30,30,30,30,17.6,6.9,39.6,27.2,13464.0,47668.0,78900.0,University of Pennsylvania
4,1,0,24,30,30,30,30,23,30,30,30,14,30,16.8,7.8,39.6,27.1,12432.0,51008.0,74000.0,Columbia University
5,1,0,30,30,30,21,22,16,30,30,30,30,30,16.8,7.3,39.5,26.8,14000.0,45800.0,70100.0,Yale University
6,1,0,30,30,30,30,19,1,30,30,30,9,30,17.2,6.7,39.6,26.8,13290.0,47243.0,76700.0,Duke University
7,1,0,30,30,30,30,27,4,30,30,30,10,30,17.3,6.5,39.1,27.1,13680.0,47286.0,71500.0,Cornell University
8,1,0,30,30,30,30,26,4,30,30,30,13,30,15.2,6.7,39.6,28.3,13620.0,41820.0,76500.0,Princeton University
9,1,0,30,30,30,30,30,2,30,30,30,24,17,17.1,6.3,38.9,27.3,14389.0,47251.0,64200.0,Northwestern University


#### Scale each column between 0 and 1 using MinMaxScaler

In [29]:
from sklearn import preprocessing

In [30]:
scaler = preprocessing.MinMaxScaler()

In [31]:
# ensure only numeric columns remain in the dataset

columns = df_onehot.columns.tolist()
columns.pop(columns.index('name'))
columns

['private',
 'public',
 'Office',
 'Shop & Service',
 'Food',
 'Arts & Entertainment',
 'Outdoors & Recreation',
 'Library',
 'College & University',
 'Nightlife Spot',
 'Residence',
 'Government Building',
 'Travel & Transport',
 'scores_engagement',
 'scores_environment',
 'scores_outcomes',
 'scores_resources',
 'stats_board',
 'stats_fees_oos',
 'stats_salary']

In [32]:
# Check for NaNs

df_onehot.isnull().values.any()

True

In [33]:
# Check for rows with NaN 

df_onehot[pd.isnull(df_onehot).any(axis=1)]

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,name
132,1,0,12,8,19,15,14,4,30,5,1,2,21,14.3,7.4,23.9,19.0,NaN,37830.0,53900.0,Art Center College of Design


In [35]:
# fill NaN with Mean
pd.set_option('max_rows', 200)

df_onehot['stats_board'] = df_onehot['stats_board'].fillna(df_onehot['stats_board'].mean())

In [36]:
# Check for NaN again

df_onehot[pd.isnull(df_onehot).any(axis=1)]

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,name


In [37]:
# Apply scaler created above

df_onehot[columns] = scaler.fit_transform(df_onehot[columns])
df_onehot.head(10)


,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,name
0,1.0,0.0,0.966667,1.0,1.0,1.0,1.000000,0.200000,1.0,0.233333,0.9,0.333333,0.500000,0.877551,0.851351,0.995671,0.850575,0.666631,0.864132,0.857914,Stanford University
1,1.0,0.0,1.000000,1.0,1.0,1.0,1.000000,0.633333,1.0,1.000000,1.0,1.000000,1.000000,0.428571,0.702703,1.000000,1.000000,0.776449,0.846331,1.000000,Harvard University
2,1.0,0.0,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.0,1.000000,1.0,1.000000,1.000000,0.530612,0.743243,0.987013,0.965517,0.623572,0.869762,0.983813,Massachusetts Institute of Technology
3,1.0,0.0,1.000000,1.0,1.0,1.0,1.000000,0.800000,1.0,1.000000,1.0,1.000000,1.000000,0.918367,0.716216,1.000000,0.844828,0.648963,0.927404,0.776978,University of Pennsylvania
4,1.0,0.0,0.800000,1.0,1.0,1.0,1.000000,0.766667,1.0,1.000000,1.0,0.466667,1.000000,0.755102,0.837838,1.000000,0.839080,0.539780,1.000000,0.688849,Columbia University
5,1.0,0.0,1.000000,1.0,1.0,0.7,0.733333,0.533333,1.0,1.000000,1.0,1.000000,1.000000,0.755102,0.770270,0.995671,0.821839,0.705671,0.886802,0.618705,Yale University
6,1.0,0.0,1.000000,1.0,1.0,1.0,0.633333,0.033333,1.0,1.000000,1.0,0.300000,1.000000,0.836735,0.689189,1.000000,0.821839,0.630554,0.918166,0.737410,Duke University
7,1.0,0.0,1.000000,1.0,1.0,1.0,0.900000,0.133333,1.0,1.000000,1.0,0.333333,1.000000,0.857143,0.662162,0.978355,0.839080,0.671815,0.919101,0.643885,Cornell University
8,1.0,0.0,1.000000,1.0,1.0,1.0,0.866667,0.133333,1.0,1.000000,1.0,0.433333,1.000000,0.428571,0.689189,1.000000,0.908046,0.665468,0.800296,0.733813,Princeton University
9,1.0,0.0,1.000000,1.0,1.0,1.0,1.000000,0.066667,1.0,1.000000,1.0,0.800000,0.535714,0.816327,0.635135,0.969697,0.850575,0.746826,0.918340,0.512590,Northwestern University


### Do KMeans cluster analysis on the processed dataset

#### Expecting Clusters to look something like this - trying with {4,5,6,7,8} clusters
:
#####  {High Ranked, Urban, Expensive, High Opportunities}
#####  {High Ranked, Semi-Urban, Less Expensive, Mid Opportunities}
##### {Mid Ranked, Urban, Expensive, High Opportunities}
##### {Mid Ranked, Semi-Urban, Less Expensive, Mid Opportunities}
##### {Mid Ranked, Sparse Pop., Moderately Cheap, Mid/Low Opportunities}
##### {Low Ranked, Urban, Expensive, Mid-to-High Opportunities}
##### {Low Ranked, Urban, Less Expensive, Low Opportunities}
##### {Low Ranked, Urban, Moderately Cheap, Low-to-Mid Opportunities}
##### {Low Ranked, Semi-urban, Cheap, Low Opportunities}


In [38]:
# set number of clusters
kclusters = 6

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_onehot[columns])

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 2, 4,
       5, 4, 4, 4, 2, 5, 2, 4, 3, 3, 5, 2, 3, 4, 0, 3, 5, 2, 5, 2, 3, 2, 3,
       0, 2, 3, 2, 5, 2, 0, 2, 3, 3, 2, 5, 2, 0, 3, 1, 2, 1, 5, 1, 5, 5, 2,
       0, 0, 0, 3, 5, 2, 0, 3, 3, 0, 1, 5, 3, 1, 3, 3, 0, 5, 3, 2, 5, 3, 0,
       0, 3, 1, 2, 1, 5, 2, 5, 5, 3, 0, 3, 3, 3, 3, 2, 3, 3, 0, 3, 0, 2, 0,
       2, 5, 2, 0, 5, 5, 0, 3, 2, 3, 5, 2, 0, 0, 0, 3, 2, 1, 2, 3, 0, 3, 1,
       0, 5, 2, 2, 3, 5, 2, 0, 2, 2, 5, 5, 0, 3, 5, 0, 5, 0, 3, 5, 0, 5, 2,
       3, 3, 2, 5, 2, 3, 3, 3, 1, 0, 2, 0, 2, 2, 0, 5, 3, 0, 1, 5, 5, 5, 2,
       0, 5, 0, 0, 3, 2, 1, 1, 3, 3, 3, 5, 1, 3, 1, 3], dtype=int32)

In [39]:
# Add cluster label to the dataset

df_onehot['Cluster'] = kmeans.labels_

In [40]:
# Inverse the Min Max Scaling

df_onehot[columns] = scaler.inverse_transform(df_onehot[columns])

In [41]:
# Add back the rank & location columns to get a full dataframe

df_onehot[['rank_order','location','Latitude','Longitude']] = df[['rank_order','location','Latitude','Longitude']]

In [42]:
df_onehot.head(10)

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,name,Cluster,rank_order,location,Latitude,Longitude
0,1.0,0.0,29.0,30.0,30.0,30.0,30.0,6.0,30.0,7.0,27.0,10.0,16.0,17.4,7.9,39.5,27.3,13631.0,44757.0,83400.0,Stanford University,4,1,California,37.427475,-122.169719
1,1.0,0.0,30.0,30.0,30.0,30.0,30.0,19.0,30.0,30.0,30.0,30.0,30.0,15.2,6.8,39.6,29.9,14669.0,43938.0,91300.0,Harvard University,4,2,Massachusetts,42.377003,-71.116660
2,1.0,0.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,15.7,7.1,39.3,29.3,13224.0,45016.0,90400.0,Massachusetts Institute of Technology,4,3,Massachusetts,42.360091,-71.094160
3,1.0,0.0,30.0,30.0,30.0,30.0,30.0,24.0,30.0,30.0,30.0,30.0,30.0,17.6,6.9,39.6,27.2,13464.0,47668.0,78900.0,University of Pennsylvania,4,4,Pennsylvania,39.952219,-75.193214
4,1.0,0.0,24.0,30.0,30.0,30.0,30.0,23.0,30.0,30.0,30.0,14.0,30.0,16.8,7.8,39.6,27.1,12432.0,51008.0,74000.0,Columbia University,4,5,New York,40.807536,-73.962573
5,1.0,0.0,30.0,30.0,30.0,21.0,22.0,16.0,30.0,30.0,30.0,30.0,30.0,16.8,7.3,39.5,26.8,14000.0,45800.0,70100.0,Yale University,4,6,Connecticut,41.316324,-72.922343
6,1.0,0.0,30.0,30.0,30.0,30.0,19.0,1.0,30.0,30.0,30.0,9.0,30.0,17.2,6.7,39.6,26.8,13290.0,47243.0,76700.0,Duke University,4,7,North Carolina,36.001426,-78.938229
7,1.0,0.0,30.0,30.0,30.0,30.0,27.0,4.0,30.0,30.0,30.0,10.0,30.0,17.3,6.5,39.1,27.1,13680.0,47286.0,71500.0,Cornell University,4,8,New York,42.453449,-76.473503
8,1.0,0.0,30.0,30.0,30.0,30.0,26.0,4.0,30.0,30.0,30.0,13.0,30.0,15.2,6.7,39.6,28.3,13620.0,41820.0,76500.0,Princeton University,4,9,New Jersey,40.343989,-74.651448
9,1.0,0.0,30.0,30.0,30.0,30.0,30.0,2.0,30.0,30.0,30.0,24.0,17.0,17.1,6.3,38.9,27.3,14389.0,47251.0,64200.0,Northwestern University,4,10,Illinois,42.056459,-87.675267


#### Finally, view these clusters on the map

In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=4)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, nam, cluster, rank in zip(df_onehot['Latitude'], df_onehot['Longitude'], df_onehot['name'], df_onehot['Cluster'], df_onehot['rank_order']):
    label = folium.Popup(str(nam) + '_Rank_' + str(rank) + '_Cluster_' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

####  Interpret Clusters; see if there are cheaper, higher ranked universities or lower ranked but cheaper & better located universities.

In [44]:
# Cluster 1

df_onehot.loc[df_onehot['Cluster']==0].describe()

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,Cluster,rank_order,Latitude,Longitude
count,35.0,35.0,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.00000,35.000000,35.000000,35.000000,35.000000,35.000000,35.0,35.000000,35.000000,35.000000
mean,1.0,0.0,29.171429,28.657143,29.942857,28.942857,29.057143,11.600000,28.742857,28.285714,27.400000,28.428571,28.457143,15.725714,5.38000,26.040000,19.497143,12569.828571,42207.857143,59037.142857,0.0,120.200000,40.732186,-83.474532
std,0.0,0.0,2.843835,2.929996,0.338062,3.105430,2.785919,9.604533,3.424332,3.602054,4.558121,3.821654,4.428138,0.995975,1.48894,2.815629,2.771014,1714.442878,4963.354737,11372.569830,0.0,44.031272,2.790884,15.739169
min,1.0,0.0,17.000000,18.000000,28.000000,17.000000,19.000000,2.000000,15.000000,15.000000,14.000000,14.000000,10.000000,13.100000,2.10000,19.200000,14.400000,8679.000000,25496.000000,40400.000000,0.0,38.000000,32.776873,-123.029907
25%,1.0,0.0,30.000000,30.000000,30.000000,30.000000,30.000000,4.000000,30.000000,29.000000,24.500000,30.000000,30.000000,15.350000,4.60000,24.400000,17.300000,11125.000000,39343.000000,51850.000000,0.0,82.500000,40.002532,-87.626078
50%,1.0,0.0,30.000000,30.000000,30.000000,30.000000,30.000000,8.000000,30.000000,30.000000,30.000000,30.000000,30.000000,15.800000,5.60000,25.500000,19.700000,12700.000000,42908.000000,55800.000000,0.0,122.000000,40.862040,-76.135116
75%,1.0,0.0,30.000000,30.000000,30.000000,30.000000,30.000000,18.500000,30.000000,30.000000,30.000000,30.000000,30.000000,16.400000,6.20000,28.100000,21.550000,13883.000000,45837.500000,61350.000000,0.0,155.000000,42.282305,-73.907513
max,1.0,0.0,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,17.500000,8.40000,34.700000,24.200000,15965.000000,49056.000000,90800.000000,0.0,188.000000,47.608970,-71.089172


In [45]:
df_onehot.loc[df_onehot['Cluster']==0]

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,name,Cluster,rank_order,location,Latitude,Longitude
37,1.0,0.0,30.0,30.0,30.0,30.0,30.0,21.0,30.0,30.0,30.0,30.0,30.0,16.9,6.4,34.7,21.2,14030.0,46664.0,60000.0,Boston University,0,38,Massachusetts,42.350500,-71.105399
46,1.0,0.0,30.0,24.0,30.0,30.0,30.0,2.0,30.0,30.0,30.0,30.0,30.0,16.0,6.2,29.8,24.2,13226.0,47972.0,49600.0,Wesleyan University,0,47,Connecticut,41.556610,-72.656904
52,1.0,0.0,30.0,30.0,30.0,30.0,23.0,3.0,27.0,26.0,30.0,30.0,30.0,16.9,5.1,28.3,24.1,11880.0,44890.0,76700.0,Lehigh University,0,53,Pennsylvania,40.606909,-75.378283
59,1.0,0.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,15.6,6.4,29.7,21.9,11700.0,46725.0,64900.0,George Washington University,0,60,District of Columbia,38.899715,-77.048599
69,1.0,0.0,30.0,27.0,30.0,30.0,30.0,10.0,30.0,30.0,25.0,30.0,30.0,15.6,6.0,26.5,23.1,12700.0,49056.0,55400.0,Trinity College,0,70,Connecticut,41.747933,-72.690334
70,1.0,0.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,17.2,6.2,29.0,18.7,14050.0,43440.0,60900.0,Northeastern University,0,71,Massachusetts,42.339807,-71.089172
71,1.0,0.0,30.0,26.0,30.0,30.0,20.0,3.0,30.0,30.0,18.0,30.0,30.0,15.8,3.5,28.7,23.0,13520.0,45635.0,69300.0,Lafayette College,0,72,Pennsylvania,40.698620,-75.208730
75,1.0,0.0,23.0,30.0,30.0,30.0,30.0,20.0,30.0,30.0,30.0,14.0,30.0,15.9,6.7,27.9,20.0,14660.0,46040.0,57100.0,Barnard College,0,76,New York,40.809097,-73.963963
78,1.0,0.0,30.0,30.0,30.0,30.0,19.0,4.0,30.0,15.0,30.0,30.0,30.0,16.4,5.3,28.8,19.7,13082.0,44222.0,78900.0,Worcester Polytechnic Institute,0,79,Massachusetts,42.274575,-71.806272
85,1.0,0.0,30.0,27.0,30.0,30.0,30.0,3.0,30.0,22.0,30.0,30.0,30.0,14.9,6.2,28.9,19.3,13620.0,47908.0,80600.0,Rensselaer Polytechnic Institute,0,86,New York,42.730172,-73.678803


In [46]:
# Cluster 2
df_onehot.loc[df_onehot['Cluster']==1].describe()

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,Cluster,rank_order,Latitude,Longitude
count,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.0,15.000000,15.000000,15.000000
mean,0.866667,0.133333,7.066667,21.266667,15.800000,7.133333,18.533333,0.800000,26.400000,6.466667,2.866667,2.000000,8.866667,15.753333,4.266667,26.020000,19.760000,11588.656616,40449.666667,51840.000000,1.0,129.800000,40.888863,-87.715693
std,0.351866,0.351866,6.261751,11.093542,8.841461,5.289702,6.423914,1.082326,9.567803,5.012366,2.474633,2.035401,6.034030,0.920300,1.574197,3.889583,2.628905,1226.915443,5001.963310,9718.083291,0.0,54.173267,3.598158,18.426427
min,0.000000,0.000000,0.000000,4.000000,5.000000,0.000000,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,14.300000,2.500000,20.900000,14.500000,9204.000000,32293.000000,39400.000000,1.0,62.000000,34.168871,-122.109879
25%,1.000000,0.000000,2.000000,9.000000,9.000000,4.000000,14.000000,0.000000,30.000000,3.000000,2.000000,0.500000,4.000000,15.050000,2.850000,23.900000,18.500000,10633.000000,36635.000000,45100.000000,1.0,81.500000,38.662232,-91.518466
50%,1.000000,0.000000,6.000000,30.000000,13.000000,5.000000,19.000000,1.000000,30.000000,6.000000,2.000000,1.000000,6.000000,15.800000,4.200000,25.500000,19.600000,11840.000000,39916.000000,49800.000000,1.0,133.000000,41.968010,-84.482172
75%,1.000000,0.000000,11.000000,30.000000,20.500000,8.000000,22.500000,1.000000,30.000000,7.500000,3.500000,3.500000,12.500000,16.100000,5.000000,27.850000,22.000000,12150.000000,44539.500000,55150.000000,1.0,185.500000,42.709241,-73.164238
max,1.000000,1.000000,20.000000,30.000000,30.000000,20.000000,30.000000,4.000000,30.000000,18.000000,9.000000,7.000000,21.000000,17.700000,7.400000,34.300000,23.300000,14102.000000,47820.000000,79000.000000,1.0,199.000000,47.751074,-69.662636


In [47]:
df_onehot.loc[df_onehot['Cluster']==1]

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,name,Cluster,rank_order,location,Latitude,Longitude
61,0.0,1.0,17.0,21.0,9.0,7.0,19.0,0.0,30.0,1.0,2.0,7.0,20.0,17.7,5.7,34.3,15.5,9204.000000,35516.0,49800.0,Michigan State University,1,62,Michigan,42.701848,-84.482172
63,1.0,0.0,2.0,25.0,10.0,6.0,14.0,1.0,30.0,4.0,2.0,2.0,6.0,16.6,4.7,27.7,23.3,12150.000000,47820.0,56000.0,Hamilton College,1,64,New York,43.052426,-75.405795
65,1.0,0.0,6.0,30.0,29.0,5.0,14.0,0.0,30.0,7.0,2.0,1.0,12.0,16.1,4.5,28.0,22.8,12150.000000,47350.0,52600.0,Colby College,1,66,Maine,44.563869,-69.662636
79,1.0,0.0,2.0,30.0,21.0,9.0,24.0,1.0,30.0,2.0,3.0,4.0,13.0,16.1,7.1,24.1,22.6,12490.000000,42656.0,44100.0,Mount Holyoke College,1,80,Massachusetts,42.253253,-72.576139
82,0.0,1.0,12.0,30.0,6.0,20.0,10.0,0.0,6.0,18.0,9.0,0.0,13.0,15.6,4.4,31.6,18.2,10344.000000,39916.0,55700.0,William & Mary,1,83,Virginia,37.090240,-95.712891
94,1.0,0.0,0.0,6.0,6.0,4.0,17.0,1.0,30.0,10.0,2.0,1.0,3.0,15.8,3.6,26.9,22.4,11560.000000,47330.0,44200.0,Kenyon College,1,95,Ohio,40.371897,-82.398308
96,1.0,0.0,0.0,5.0,12.0,4.0,19.0,1.0,30.0,1.0,2.0,1.0,4.0,14.5,3.7,30.6,19.3,12057.000000,41283.0,79000.0,Rose-Hulman Institute of Technology,1,97,Indiana,39.482839,-87.324040
132,1.0,0.0,12.0,8.0,19.0,15.0,14.0,4.0,30.0,5.0,1.0,2.0,21.0,14.3,7.4,23.9,19.0,12126.849246,37830.0,53900.0,Art Center College of Design,1,133,California,34.168871,-118.185157
137,1.0,0.0,8.0,30.0,30.0,5.0,24.0,1.0,30.0,7.0,7.0,4.0,4.0,16.1,5.3,21.2,21.6,11840.000000,46423.0,45200.0,Wheaton College (Massachusetts),1,138,Massachusetts,41.968010,-71.185194
169,1.0,0.0,9.0,30.0,18.0,5.0,19.0,1.0,30.0,6.0,5.0,1.0,5.0,15.2,2.9,24.2,19.6,10930.000000,35970.0,46000.0,Saint Mary's College,1,170,Indiana,37.841625,-122.109879


In [48]:
# Cluster 3

df_onehot.loc[df_onehot['Cluster']==2].describe()

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,Cluster,rank_order,Latitude,Longitude
count,40.0,40.0,40.0000,40.000000,40.000000,40.00000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.0,40.000000,40.000000,40.000000
mean,0.0,1.0,27.2000,28.275000,29.850000,28.57500,26.450000,4.600000,29.775000,28.400000,27.650000,17.500000,27.900000,16.527500,6.170000,29.880000,16.585000,11025.825000,30685.300000,53737.500000,2.0,103.425000,38.706765,-90.941677
std,0.0,0.0,5.3407,4.119917,0.699817,2.94294,6.452787,3.953577,1.049725,3.248668,5.146819,8.174381,4.650889,0.807334,1.611847,5.042801,2.463742,1729.009556,5064.514325,6848.083674,0.0,51.601971,4.377518,16.240434
min,0.0,1.0,11.0000,16.000000,26.000000,19.00000,0.000000,0.000000,24.000000,19.000000,10.000000,5.000000,14.000000,14.000000,3.000000,20.800000,12.500000,8270.000000,20810.000000,42700.000000,2.0,22.000000,29.643633,-122.303520
25%,0.0,1.0,27.0000,30.000000,30.000000,29.75000,23.750000,1.750000,30.000000,29.750000,30.000000,10.000000,29.500000,16.275000,4.725000,26.200000,14.925000,9932.500000,27888.250000,48975.000000,2.0,53.500000,35.507521,-99.612938
50%,0.0,1.0,30.0000,30.000000,30.000000,30.00000,30.000000,3.000000,30.000000,30.000000,30.000000,16.500000,30.000000,16.750000,5.850000,28.900000,16.500000,10716.500000,30086.000000,53150.000000,2.0,111.000000,39.879297,-85.454615
75%,0.0,1.0,30.0000,30.000000,30.000000,30.00000,30.000000,7.250000,30.000000,30.000000,30.000000,23.500000,30.000000,17.100000,7.425000,34.150000,18.100000,11883.500000,33534.250000,56925.000000,2.0,145.500000,41.652686,-78.762251
max,0.0,1.0,30.0000,30.000000,30.000000,30.00000,30.000000,14.000000,30.000000,30.000000,30.000000,30.000000,30.000000,17.600000,9.000000,38.900000,22.200000,15830.000000,43377.000000,75200.000000,2.0,190.000000,47.655335,-72.253981


In [49]:
df_onehot.loc[df_onehot['Cluster']==2]

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,name,Cluster,rank_order,location,Latitude,Longitude
21,0.0,1.0,20.0,30.0,30.0,27.0,30.0,2.0,30.0,30.0,19.0,25.0,15.0,17.4,6.0,38.9,22.2,10246.0,43377.0,57600.0,University of Michigan,2,22,Michigan,42.278044,-83.738224
27,0.0,1.0,17.0,30.0,30.0,24.0,30.0,4.0,30.0,30.0,30.0,28.0,30.0,16.5,8.9,37.3,20.0,14511.0,35583.0,59400.0,"University of California, Los Angeles",2,28,California,34.068921,-118.445181
29,0.0,1.0,30.0,16.0,30.0,19.0,16.0,3.0,30.0,23.0,30.0,23.0,30.0,16.9,5.3,37.4,21.7,10592.0,33418.0,50700.0,University of North Carolina at Chapel Hill,2,30,North Carolina,35.904912,-79.046913
34,0.0,1.0,30.0,30.0,30.0,30.0,0.0,13.0,30.0,30.0,22.0,30.0,26.0,16.8,8.4,36.3,18.1,15830.0,35850.0,61700.0,"University of California, Berkeley",2,35,California,37.870921,-122.268803
40,0.0,1.0,30.0,30.0,30.0,30.0,30.0,2.0,30.0,30.0,30.0,7.0,30.0,17.2,5.6,34.1,21.3,10030.0,28804.0,53400.0,Purdue University,2,41,Indiana,40.423705,-86.921195
42,0.0,1.0,30.0,30.0,30.0,30.0,20.0,0.0,30.0,30.0,30.0,13.0,30.0,17.1,8.4,33.5,18.3,14316.0,36774.0,56700.0,"University of California, Davis",2,43,California,38.537381,-121.752801
44,0.0,1.0,30.0,30.0,30.0,30.0,26.0,6.0,30.0,30.0,30.0,19.0,30.0,17.1,7.4,37.2,15.4,10848.0,32621.0,56600.0,University of Illinois at Urbana-Champaign,2,45,Illinois,40.109888,-88.228383
47,0.0,1.0,16.0,30.0,30.0,30.0,23.0,3.0,30.0,30.0,26.0,12.0,23.0,15.1,8.6,34.3,17.9,12254.0,36305.0,59300.0,"University of California, San Diego",2,48,California,32.880060,-117.234014
49,0.0,1.0,30.0,30.0,30.0,30.0,23.0,4.0,30.0,19.0,30.0,9.0,30.0,15.5,4.9,36.6,17.8,10052.0,42394.0,59300.0,University of Virginia,2,50,Virginia,38.033553,-78.507977
51,0.0,1.0,30.0,30.0,30.0,30.0,30.0,10.0,30.0,30.0,30.0,30.0,30.0,15.3,7.1,36.4,15.5,10434.0,30698.0,74200.0,Georgia Institute of Technology,2,52,Georgia,33.775618,-84.396285


In [50]:
# Cluster 4

df_onehot.loc[df_onehot['Cluster']==3].describe()

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,Cluster,rank_order,Latitude,Longitude
count,47.0,47.0,47.000000,47.000000,47.000000,47.00000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.0,47.000000,47.000000,47.000000
mean,1.0,0.0,29.659574,28.340426,29.085106,28.87234,26.319149,2.702128,27.978723,27.404255,26.510638,11.340426,28.978723,16.178723,5.112766,25.323404,21.072340,12379.191489,42212.127660,53678.723404,3.0,113.553191,38.609479,-91.588821
std,0.0,0.0,1.936790,3.583031,2.338987,3.82564,5.320618,2.358287,3.559979,5.265390,6.275807,6.037119,3.293534,0.847999,1.549981,4.089925,2.777448,1707.104307,7709.504181,7909.321308,0.0,51.077864,4.927669,18.306744
min,1.0,0.0,17.000000,15.000000,21.000000,10.00000,12.000000,0.000000,19.000000,11.000000,11.000000,2.000000,14.000000,14.000000,1.600000,16.500000,13.700000,7330.000000,5000.000000,35700.000000,3.0,32.000000,25.719168,-122.630372
25%,1.0,0.0,30.000000,30.000000,30.000000,30.00000,23.500000,1.000000,27.500000,28.000000,27.000000,5.500000,30.000000,15.600000,4.050000,23.200000,18.700000,11204.000000,40193.000000,48700.000000,3.0,77.500000,34.902348,-108.305606
50%,1.0,0.0,30.000000,30.000000,30.000000,30.00000,30.000000,2.000000,30.000000,30.000000,30.000000,11.000000,30.000000,16.200000,5.000000,25.400000,21.700000,12582.000000,44350.000000,53400.000000,3.0,107.000000,39.739300,-88.859591
75%,1.0,0.0,30.000000,30.000000,30.000000,30.00000,30.000000,4.000000,30.000000,30.000000,30.000000,16.000000,30.000000,16.700000,6.400000,27.550000,23.200000,13668.000000,47112.500000,56700.000000,3.0,159.500000,42.278667,-76.064922
max,1.0,0.0,30.000000,30.000000,30.000000,30.00000,30.000000,11.000000,30.000000,30.000000,30.000000,23.000000,30.000000,18.000000,7.800000,33.200000,25.000000,14950.000000,49570.000000,75900.000000,3.0,200.000000,47.261764,-69.963997


In [51]:
df_onehot.loc[df_onehot['Cluster']==3]

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,name,Cluster,rank_order,location,Latitude,Longitude
31,1.0,0.0,30.0,30.0,30.0,30.0,23.0,2.0,30.0,28.0,30.0,9.0,30.0,16.4,7.6,31.4,25.0,14700.0,45832.0,52000.0,Pomona College,3,32,California,34.097743,-117.711803
32,1.0,0.0,30.0,30.0,30.0,30.0,30.0,3.0,30.0,30.0,23.0,20.0,30.0,17.1,6.8,31.7,24.1,14950.0,44724.0,43500.0,Smith College,3,33,Massachusetts,42.318096,-72.637234
35,1.0,0.0,30.0,30.0,30.0,30.0,23.0,4.0,30.0,30.0,30.0,8.0,30.0,16.0,5.8,33.2,24.7,14350.0,47214.0,56100.0,Haverford College,3,36,Pennsylvania,40.009348,-75.305656
38,1.0,0.0,30.0,30.0,30.0,30.0,16.0,3.0,30.0,30.0,30.0,4.0,30.0,15.1,7.0,31.8,24.3,14350.0,45540.0,52000.0,Bryn Mawr College,3,39,Pennsylvania,40.028106,-75.315234
43,1.0,0.0,30.0,30.0,30.0,30.0,16.0,0.0,30.0,30.0,30.0,5.0,30.0,17.5,7.8,29.8,22.1,12684.0,44350.0,53400.0,University of Miami,3,44,Florida,25.719168,-80.277125
45,1.0,0.0,30.0,23.0,30.0,30.0,30.0,1.0,30.0,30.0,15.0,12.0,30.0,16.4,5.1,31.3,24.0,12760.0,46808.0,55000.0,Bowdoin College,3,46,Maine,43.907693,-69.963997
48,1.0,0.0,30.0,30.0,30.0,30.0,25.0,2.0,30.0,30.0,30.0,11.0,30.0,16.7,6.6,27.7,24.1,14820.0,47395.0,66400.0,Claremont McKenna College,3,49,California,34.101777,-117.707922
54,1.0,0.0,30.0,30.0,21.0,30.0,30.0,3.0,23.0,30.0,30.0,4.0,30.0,16.2,4.3,30.6,23.0,12638.0,46200.0,60400.0,Wake Forest University,3,55,North Carolina,36.135250,-80.276342
55,1.0,0.0,30.0,30.0,30.0,10.0,27.0,10.0,30.0,19.0,30.0,13.0,30.0,16.9,4.9,27.4,24.8,12556.0,48306.0,52600.0,Tulane University,3,56,Louisiana,29.940348,-90.120728
60,1.0,0.0,30.0,23.0,30.0,30.0,24.0,5.0,28.0,30.0,30.0,18.0,30.0,16.3,5.7,26.8,24.7,13708.0,46960.0,55900.0,University of Rochester,3,61,New York,43.130553,-77.626003


In [52]:
# Cluster 5

df_onehot.loc[df_onehot['Cluster']==4].describe()

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,Cluster,rank_order,Latitude,Longitude
count,26.0,26.0,26.000000,26.000000,26.0,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.0,26.000000,26.000000,26.000000
mean,1.0,0.0,28.384615,29.576923,30.0,28.153846,27.615385,9.500000,29.346154,28.000000,28.153846,21.423077,26.807692,16.515385,6.750000,37.942308,26.623077,13582.500000,46175.153846,70461.538462,4.0,14.807692,39.329671,-83.675456
std,0.0,0.0,4.186425,1.963122,0.0,3.946566,4.000769,7.617086,3.333974,5.230679,3.936418,8.438830,5.276508,0.912005,0.837735,2.201486,1.579951,961.931276,2392.771342,9608.665960,0.0,9.537376,3.414323,14.815935
min,1.0,0.0,13.000000,20.000000,30.0,16.000000,19.000000,1.000000,13.000000,7.000000,16.000000,8.000000,14.000000,14.100000,4.300000,32.000000,23.600000,11994.000000,40566.000000,56500.000000,4.0,1.000000,29.717394,-122.169719
25%,1.0,0.0,30.000000,30.000000,30.0,30.000000,26.250000,4.000000,30.000000,30.000000,30.000000,13.250000,25.000000,15.975000,6.325000,37.450000,25.825000,12994.500000,44819.750000,62725.000000,4.0,7.250000,37.732803,-87.399699
50%,1.0,0.0,30.000000,30.000000,30.0,30.000000,30.000000,7.500000,30.000000,30.000000,30.000000,22.500000,30.000000,16.750000,6.900000,39.000000,26.800000,13432.000000,46605.500000,68750.000000,4.0,13.500000,40.586161,-78.005244
75%,1.0,0.0,30.000000,30.000000,30.0,30.000000,30.000000,11.750000,30.000000,30.000000,30.000000,30.000000,30.000000,17.200000,7.250000,39.475000,27.275000,14167.250000,47609.500000,76350.000000,4.0,20.750000,41.817231,-73.971045
max,1.0,0.0,30.000000,30.000000,30.0,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,17.600000,7.900000,39.600000,29.900000,16782.000000,51008.000000,91300.000000,4.0,37.000000,42.453449,-71.094160


In [53]:
df_onehot.loc[df_onehot['Cluster']==4]

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,name,Cluster,rank_order,location,Latitude,Longitude
0,1.0,0.0,29.0,30.0,30.0,30.0,30.0,6.0,30.0,7.0,27.0,10.0,16.0,17.4,7.9,39.5,27.3,13631.0,44757.0,83400.0,Stanford University,4,1,California,37.427475,-122.169719
1,1.0,0.0,30.0,30.0,30.0,30.0,30.0,19.0,30.0,30.0,30.0,30.0,30.0,15.2,6.8,39.6,29.9,14669.0,43938.0,91300.0,Harvard University,4,2,Massachusetts,42.377003,-71.116660
2,1.0,0.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,15.7,7.1,39.3,29.3,13224.0,45016.0,90400.0,Massachusetts Institute of Technology,4,3,Massachusetts,42.360091,-71.094160
3,1.0,0.0,30.0,30.0,30.0,30.0,30.0,24.0,30.0,30.0,30.0,30.0,30.0,17.6,6.9,39.6,27.2,13464.0,47668.0,78900.0,University of Pennsylvania,4,4,Pennsylvania,39.952219,-75.193214
4,1.0,0.0,24.0,30.0,30.0,30.0,30.0,23.0,30.0,30.0,30.0,14.0,30.0,16.8,7.8,39.6,27.1,12432.0,51008.0,74000.0,Columbia University,4,5,New York,40.807536,-73.962573
5,1.0,0.0,30.0,30.0,30.0,21.0,22.0,16.0,30.0,30.0,30.0,30.0,30.0,16.8,7.3,39.5,26.8,14000.0,45800.0,70100.0,Yale University,4,6,Connecticut,41.316324,-72.922343
6,1.0,0.0,30.0,30.0,30.0,30.0,19.0,1.0,30.0,30.0,30.0,9.0,30.0,17.2,6.7,39.6,26.8,13290.0,47243.0,76700.0,Duke University,4,7,North Carolina,36.001426,-78.938229
7,1.0,0.0,30.0,30.0,30.0,30.0,27.0,4.0,30.0,30.0,30.0,10.0,30.0,17.3,6.5,39.1,27.1,13680.0,47286.0,71500.0,Cornell University,4,8,New York,42.453449,-76.473503
8,1.0,0.0,30.0,30.0,30.0,30.0,26.0,4.0,30.0,30.0,30.0,13.0,30.0,15.2,6.7,39.6,28.3,13620.0,41820.0,76500.0,Princeton University,4,9,New Jersey,40.343989,-74.651448
9,1.0,0.0,30.0,30.0,30.0,30.0,30.0,2.0,30.0,30.0,30.0,24.0,17.0,17.1,6.3,38.9,27.3,14389.0,47251.0,64200.0,Northwestern University,4,10,Illinois,42.056459,-87.675267


In [54]:
# Cluster 6

df_onehot.loc[df_onehot['Cluster']==5].describe()

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,Cluster,rank_order,Latitude,Longitude
count,37.0,37.0,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.0,37.000000,37.000000,37.000000
mean,1.0,0.0,18.864865,26.729730,24.810811,22.729730,28.648649,1.378378,29.432432,22.486486,13.378378,7.621622,19.135135,16.083784,4.327027,27.113514,20.975676,11772.864865,43790.756757,54424.324324,5.0,110.459459,40.791575,-80.906635
std,0.0,0.0,8.192687,6.016868,6.552514,7.101051,3.831276,1.036858,2.291779,7.733842,7.576834,3.882663,7.036264,0.750745,1.510270,4.297555,2.652295,1575.627091,4827.323151,10637.392261,0.0,53.776593,2.979845,9.484497
min,1.0,0.0,3.000000,12.000000,9.000000,0.000000,14.000000,0.000000,20.000000,5.000000,3.000000,1.000000,7.000000,15.000000,1.800000,20.800000,15.800000,8724.000000,28050.000000,39300.000000,5.0,15.000000,34.041404,-118.709581
25%,1.0,0.0,12.000000,30.000000,21.000000,18.000000,30.000000,1.000000,30.000000,16.000000,8.000000,5.000000,14.000000,15.400000,3.200000,24.600000,19.200000,10790.000000,41700.000000,47100.000000,5.0,67.000000,39.841124,-84.781506
50%,1.0,0.0,21.000000,30.000000,30.000000,23.000000,30.000000,1.000000,30.000000,26.000000,12.000000,8.000000,17.000000,16.000000,4.600000,25.600000,20.900000,11920.000000,45617.000000,51300.000000,5.0,117.000000,40.954687,-77.540974
75%,1.0,0.0,26.000000,30.000000,30.000000,30.000000,30.000000,2.000000,30.000000,30.000000,18.000000,10.000000,26.000000,16.600000,5.300000,28.700000,23.300000,13116.000000,47436.000000,60300.000000,5.0,155.000000,42.816615,-75.160881
max,1.0,0.0,30.000000,30.000000,30.000000,30.000000,30.000000,4.000000,30.000000,30.000000,30.000000,16.000000,30.000000,17.700000,7.200000,39.200000,26.700000,14494.000000,48682.000000,86100.000000,5.0,196.000000,44.663761,-71.080628


In [55]:
df_onehot.loc[df_onehot['Cluster']==5]

,private,public,Office,Shop & Service,Food,Arts & Entertainment,Outdoors & Recreation,Library,College & University,Nightlife Spot,Residence,Government Building,Travel & Transport,scores_engagement,scores_environment,scores_outcomes,scores_resources,stats_board,stats_fees_oos,stats_salary,name,Cluster,rank_order,location,Latitude,Longitude
14,1.0,0.0,21.0,30.0,24.0,28.0,30.0,1.0,29.0,14.0,10.0,4.0,26.0,16.8,6.0,39.2,26.7,13839.0,48108.0,67400.0,Dartmouth College,5,15,New Hampshire,43.704441,-72.288694
23,1.0,0.0,7.0,30.0,30.0,30.0,30.0,2.0,30.0,20.0,11.0,3.0,15.0,15.7,6.6,37.5,24.6,12760.0,48310.0,56100.0,Williams College,5,24,Massachusetts,42.712958,-73.203640
28,1.0,0.0,30.0,20.0,30.0,17.0,30.0,3.0,30.0,5.0,6.0,9.0,20.0,16.5,7.2,34.5,24.4,13960.0,45078.0,56600.0,Wellesley College,5,29,Massachusetts,42.293573,-71.305928
33,1.0,0.0,21.0,30.0,20.0,22.0,30.0,1.0,30.0,21.0,18.0,6.0,30.0,17.7,6.6,30.9,24.6,13550.0,46060.0,48900.0,Swarthmore College,5,34,Pennsylvania,39.904035,-75.352317
39,1.0,0.0,24.0,30.0,17.0,30.0,30.0,1.0,30.0,30.0,10.0,8.0,26.0,16.7,5.8,32.2,23.5,12366.0,47736.0,47800.0,Carleton College,5,40,Minnesota,44.461365,-93.155826
41,1.0,0.0,15.0,30.0,30.0,20.0,30.0,2.0,30.0,19.0,6.0,7.0,10.0,16.8,4.9,32.6,23.7,13116.0,46044.0,54200.0,Middlebury College,5,42,Vermont,44.008260,-73.177315
50,1.0,0.0,27.0,30.0,18.0,30.0,30.0,2.0,30.0,30.0,13.0,8.0,24.0,16.7,3.8,31.2,23.0,11642.0,48498.0,68900.0,Bucknell University,5,51,Pennsylvania,40.954687,-76.883551
57,1.0,0.0,5.0,30.0,30.0,21.0,30.0,1.0,30.0,26.0,6.0,1.0,7.0,16.0,5.2,28.7,23.6,11970.0,48175.0,60900.0,Colgate University,5,58,New York,42.816615,-75.540176
64,1.0,0.0,30.0,30.0,30.0,28.0,30.0,4.0,30.0,26.0,13.0,2.0,16.0,15.4,4.6,29.2,22.5,12769.0,45377.0,55100.0,Davidson College,5,65,North Carolina,35.500802,-80.844673
66,1.0,0.0,9.0,30.0,30.0,30.0,30.0,2.0,30.0,16.0,11.0,13.0,16.0,15.4,4.8,28.0,23.3,13106.0,48682.0,39300.0,Oberlin College,5,67,Ohio,41.293939,-82.217379
